# Генерация данных по продажам а-ля qm

![](ord_prod_bd_scheme.png)

Входные данные: только таблица products: айди, название, категория, цена, затраты, частота покупок.

Создать список id продуктов с повторами в соответствии с частотой покупок - product_ids.

Функция create_new_users(last_user_id, number_of_new_users=250):
__генерим number_of_new_users *= (1 + 0.25 * снр)

В первый месяц создать 250± пользователей. 

В последующие месяцы в начале месяца:
- имитируем отток пользователей. Шафлим список юзеров. Убираем долю± клиентов (50%);
- добавлять так, чтобы было +250± человек ежемесячно. Тогда рост базы будет постепенно уменьшаться в процентах. Создаем список новых пользователей, шафлим и какие-то слева элементы добавляем еще раз, чтобы сымитировать разное время жизни клиентов. 0.04 - x7, 0.08 - x5, 0.11 - x3, 0.17 - x2. Получится, что долго живущие клиенты и покупать будут чаще, во всяком случае в начале своего жизненного пути.

Генерируем какое количество покупок в день случится. Число пользователей * среднее базовое число покупок в месяц / 30 ±.  
__Выбираем случайного пользователя из списка.  
____Количество продуктов ~~по остатку при делении id ± вычисляем. %11: <1 - 1, <3 - 2, <6 - 3, <9 - 4, <10 - 5, <11 - 6.~~ Пока решил рандомно: [1, 2, 3], p=[0.52, 0.36, 0.12].  
______Выбираем случайный продукт из списка и генерим случайное количество продуктов [1, 2, 3, 4, 5], [0.58, 0.28, 0.11, 0.02, 0.01]  

In [22]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

## Создание справочников
На основе исходного файла создадим таблицы-справочники:

In [2]:
df = pd.read_excel('./data_files/products_original.xlsx')
df.head()

,product_id,product_name,category,price,cost,freq
0,8,Гномье кольцо,артефакторика,450,300,1
1,25,Ёлка-палка,артефакторика,300,200,1
2,23,Жало,артефакторика,650,450,2
3,13,Кольцо нибелунга,артефакторика,800,550,2
4,9,Нарсил,артефакторика,700,450,1


In [12]:
df[['product_id', 'price']].to_csv('./data_files/prices.csv', index=False)

In [13]:
df[['product_id', 'cost']].to_csv('./data_files/costs.csv', index=False)

In [21]:
df['category_id'] = df['category'].map({'артефакторика': 1, 'зелья': 2, 'яства': 3})

In [22]:
df[['product_id', 'product_name', 'category_id']].to_csv('./data_files/products.csv', index=False)

In [ ]:
df[['category_id', 'category']].drop_duplicates().to_csv('./data_files/categories.csv', index=False)

,category_id,category
0,1,артефакторика
7,2,зелья
13,3,яства


## Функции

In [89]:
def create_new_users(user_ids: np.array, last_user_id: int, number_of_new_users: int=250, volatility=0.3) -> int:
    number_of_new_users += int(number_of_new_users * volatility * random.normalvariate())
    if number_of_new_users < 0:
        number_of_new_users %= 100
    new_users_list = np.array([*range(last_user_id + 1, last_user_id + number_of_new_users + 1)])
    user_ids = np.append(user_ids, new_users_list)
    np.random.shuffle(new_users_list)
    for share in [0.48, 0.33, 0.21, 0.12, 0.04, 0.02]:
        user_ids = np.append(user_ids, new_users_list[:int(share * number_of_new_users)])
    last_user_id += number_of_new_users
    return last_user_id, user_ids

In [90]:
def churn_users(user_ids: np.array, churn_rate=0.25, volatility=0.3):
    users_id_length = user_ids.size
    churn = int(churn_rate * users_id_length)
    churn += int(churn * volatility * random.normalvariate())
    if churn < 0:
        churn %= 100
    np.random.shuffle(user_ids)
    user_ids = user_ids[churn:]
    return churn, user_ids

In [91]:
def create_order(current_day, order_id, user_id):
    quantity_of_products = np.random.choice(a=[1, 2, 3, 4, 5], p=[0.58, 0.28, 0.11, 0.02, 0.01])
    for _ in range(quantity_of_products):
        product_id = np.random.choice(product_ids)
        quantity = np.random.choice(a=[1, 2, 3], p=[0.52, 0.36, 0.12])
        orders.loc[orders.shape[0]] = [order_id, user_id, current_day, product_id, quantity]

In [92]:
def create_next_day_orders(previous_date, last_order_id: int, purchases_frequency=2, volatility=0.3):
    current_day = previous_date + pd.Timedelta(1, 'day')
    number_of_purchases = int(user_ids.size * purchases_frequency / (2.57 * 30))  # в среднем 2 покупки в месяц, но надо учиывать повторы юзеров в моей генерации
    number_of_purchases += int(number_of_purchases * volatility * random.normalvariate())
    if number_of_purchases < 0:
        number_of_purchases = 0
    for i in range(number_of_purchases):
        # print('cycle')
        last_order_id += 1
        user_id = np.random.choice(user_ids)
        create_order(current_day, last_order_id, user_id)
    # print('cndo')
    return current_day, last_order_id

Создадим список айдишников товаров, в котором они будут повторяться по несколько раз в зависимости от частоты покупок этого товара:

In [17]:
df = pd.read_excel('./data_files/products_original.xlsx')

product_ids = []
for id, freq in zip(df['product_id'], df['freq']):
    product_ids.extend([id] * freq)

product_ids = np.array(product_ids)
product_ids[:10]

array([ 8, 25, 23, 23, 13, 13,  9, 12, 22, 22])

In [18]:
df.head()

,product_id,product_name,category,price,cost,freq
0,8,Гномье кольцо,артефакторика,450,300,1
1,25,Ёлка-палка,артефакторика,300,200,1
2,23,Жало,артефакторика,650,450,2
3,13,Кольцо нибелунга,артефакторика,800,550,2
4,9,Нарсил,артефакторика,700,450,1


In [135]:
orders = pd.DataFrame({
    'order_id': [0], 
    'user_id': [0], 
    'order_date': ['2019-12-31 00:00:00'], 
    'product_id': [0], 
    'quantity': [0]
})

In [136]:
orders.order_date = pd.to_datetime(orders.order_date)

In [ ]:
user_ids = np.array([1])
last_user_id = 1
last_order_id = 0
number_of_days = 365 * 3 + 1
period_for_new_users = 37
current_day = pd.to_datetime('2019-12-31')

In [153]:
for day in tqdm(range(number_of_days)):
    if day % period_for_new_users == 0:
        count_before_churn = len(set(user_ids))
        churn, user_ids = churn_users(user_ids)
        count_after_churn = len(set(user_ids))
        last_user_id, user_ids = create_new_users(user_ids, last_user_id, number_of_new_users=count_before_churn-count_after_churn+250)
    current_day, last_order_id = create_next_day_orders(current_day, last_order_id)
# current_day, last_order_id

100%|██████████| 365/365 [2:47:33<00:00, 27.55s/it]  


In [154]:
orders

,order_id,user_id,order_date,product_id,quantity
0,0,0,2019-12-31,0,0
1,1,80,2020-01-01,3,1
2,1,80,2020-01-01,14,1
3,2,146,2020-01-01,4,3
4,3,11,2020-01-01,14,2
...,...,...,...,...,...
509445,317859,41789,2023-12-31,4,1
509446,317859,41789,2023-12-31,6,2
509447,317860,39495,2023-12-31,14,3
509448,317861,37047,2023-12-31,8,1


In [157]:
len(set(user_ids))

9322

In [ ]:
orders[1:].to_csv('data_ignored/orders.csv', index=False)

In [150]:
orders.quantity.value_counts()

quantity
1    150874
2    103901
3     34659
0         1
Name: count, dtype: int64